In [1]:
import os
import openai
import json
import pandas as pd

openai.api_key = 'OPEN_API_KEY'


In [2]:
public_art = pd.read_csv('/Users/samedelstein/Downloads/Syracuse_Public_Art.csv')

public_art = public_art.rename(columns={"X": "longitude", "Y": "latitude"}, errors="raise").sample(40)
public_art_string = public_art.to_string()


In [3]:
question = """based on the following csv, please suggest what a  new  work of syracuse public art  would be and recommend and list out the following information - only list this information do not give me an introduction about the piece:
- name
- location
- latitude
- longitude
- material used to create the piece
- description of what the piece looks like"""

question + public_art_string


"based on the following csv, please suggest what a  new  work of syracuse public art  would be and recommend and list out the following information - only list this information do not give me an introduction about the piece:\n- name\n- location\n- latitude\n- longitude\n- material used to create the piece\n- description of what the piece looks like    longitude   latitude  FID                                                Title       Type                                                              Address   Latitude  Longitude Artist_Last_ Artist_First                        Additional_Artists                                                                           Media  Year_Created  Year_Erected   TNT_Area           Neighborhood                                                                        Specific_Location                                                                                                                                  Image_Url\n46 -76.155051  43.049900  

In [4]:
response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant that recommends and lists the name, location, latitude, longitude, type of material, and description of a new art piece in Syracuse, NY."},
        {"role": "user", "content": question + public_art_string},
        {"role": "user", "content": 'Please recommend and list the name, location, latitude, longitude, type of material, and description'}

    ]
)

In [5]:
response["choices"][0]["message"]["content"]

"Based on the given csv, I recommend a new work of public art in the Syracuse, NY area. \n\nName: Steel Grace\n\nLocation: Thornden Park\n\nLatitude: 43.0383\n\nLongitude: -76.1277\n\nMaterial: Stainless steel\n\nDescription: This sculpture will be made of stainless steel and will be located in Syracuse's historic Thornden Park. The sculpture features a spiral design that is both modern and elegant. The spiral design represents the interconnectedness of all things and is meant to convey a sense of unity and harmony. The sculpture will be installed on a stone base and will be meant to be viewed from all angles. Visitors to Thornden Park will be able to enjoy this beautiful piece of public art and contemplate the many meanings and interpretations of the steel grace sculpture."

In [6]:
image_response = openai.Image.create(
  prompt=response["choices"][0]["message"]["content"],
  n=1,
  size="1024x1024"
)
image_url = image_response['data'][0]['url']

In [7]:
image_url

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-lCOjkmfSz3vJU4xBDIilXaGH/user-zrBjiQr0a0ZqPz3AvA33GnY6/img-7a298MfxrtM8COh4noEa9v8C.png?st=2023-03-11T17%3A00%3A15Z&se=2023-03-11T19%3A00%3A15Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-03-11T17%3A45%3A27Z&ske=2023-03-12T17%3A45%3A27Z&sks=b&skv=2021-08-06&sig=t8J2J%2BdKhk/36KEFyzqGKwWYQSQrKyr8n/DxUC6R6J0%3D'

In [8]:
chunks = response["choices"][0]["message"]["content"].split('\n')

df = pd.DataFrame(chunks)
df[['Type','Description']] = df[0].str.split(':',expand=True)


In [9]:
df

,0,Type,Description
0,"Based on the given csv, I recommend a new work...","Based on the given csv, I recommend a new work...",None
1,,,None
2,Name: Steel Grace,Name,Steel Grace
3,,,None
4,Location: Thornden Park,Location,Thornden Park
5,,,None
6,Latitude: 43.0383,Latitude,43.0383
7,,,None
8,Longitude: -76.1277,Longitude,-76.1277
9,,,None


In [ ]:
df = df.replace("<br>"," ", regex=True)
df.to_csv('public_art.csv')

In [10]:
Longitude = float(df['Description'][df['Type']=='Longitude'].iloc[0])
Latitude = float(df['Description'][df['Type']=='Latitude'].iloc[0])


In [11]:
st_map = pd.DataFrame([[Latitude, Longitude]], columns=['lat', 'lon'])

In [14]:
df[df['Type']=='Name']['Description'].iloc[0]

' Steel Grace'

In [15]:
df[df['Type']=='Location']['Description'].iloc[0]

' Thornden Park'

In [16]:
df[df['Type']=='Material']['Description'].iloc[0]

' Stainless steel'

In [17]:
df[df['Type']=='Description']['Description'].iloc[0]

" This sculpture will be made of stainless steel and will be located in Syracuse's historic Thornden Park. The sculpture features a spiral design that is both modern and elegant. The spiral design represents the interconnectedness of all things and is meant to convey a sense of unity and harmony. The sculpture will be installed on a stone base and will be meant to be viewed from all angles. Visitors to Thornden Park will be able to enjoy this beautiful piece of public art and contemplate the many meanings and interpretations of the steel grace sculpture."